This notebook relies on the following cmd line tools, ensure they are installed and in the system
- tippecanoe
- mapshaper
- aws cli

In [2]:
from pathlib import Path
import sys
import logging

from IPython.lib import backgroundjobs as bg

scripts_dir = Path('..').joinpath('src')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from pipelines.pipes import get_pipes, execution_order, filter_pipes

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("fiona").setLevel(logging.WARNING)
jobs = bg.BackgroundJobManager()

In [2]:
# def execute_pipes(pipes):
#     ordered_pipes = execution_order(pipes)
#     for pipe in ordered_pipes:
#         jobs.new(pipe().execute)

# # this code will execute all pipelines selected in background.

# mypipes_subset = filter_pipes(get_pipes().items(), ['eez_tiles'])
# execute_pipes(mypipes_subset)

In [3]:
get_pipes()

{'eez_intermediate': pipelines.intermediate_pipes.EEZIntermediatePipe.EEZIntermediatePipe,
 'mpaatlas_intermediate': pipelines.intermediate_pipes.MpaAtlasIntermediatePipe.MpaAtlasIntermediatePipe,
 'mpa_intermediate': pipelines.intermediate_pipes.MpasIntermediatePipe.MpasIntermediatePipe,
 'protectedseas_intermediate': pipelines.intermediate_pipes.ProtectedSeasIntermediatePipe.ProtectedSeasIntermediatePipe,
 'eez_tiles': pipelines.tiles_pipes.EEZTilesPipe.EEZTilesPipe,
 'mpaatlas_tiles': pipelines.tiles_pipes.MpaAtlasTilesPipe.MpaAtlasTilesPipe,
 'mpa_tiles': pipelines.tiles_pipes.MpasTilesPipe.MpasTilesPipe,
 'protectedseas_tiles': pipelines.tiles_pipes.ProtectedSeasTilesPipe.ProtectedSeasTilesPipe,
 'regions_tiles': pipelines.tiles_pipes.RegionsTilesPipe.RegionsTilesPipe,
 'habitats_precalc': pipelines.precalc_pipes.HabitatsPipe.HabitatsStatsPipe,
 'lfp_precalc': pipelines.precalc_pipes.LFPPipe.LFPStatsPipe,
 'eez_locations_precalc': pipelines.precalc_pipes.LocationsPipe.LocationsPip

In [4]:
mypipes_subset = filter_pipes(get_pipes(), ["mpaatlas_intermediate"])

In [5]:
# for n, pipe in mypipes_subset.items():
#     new_pipe = pipe()
#     new_pipe.extract().transform()

In [6]:
#!micromamba install pandera pandera-hypotheses pandera-io pandera-geopandas -c conda-forge -y

In [4]:
from pathlib import Path
from typing import Union
import pandas as pd
import geopandas as gpd
import numpy as np

from pipelines.utils import load_regions

from pipelines.output_schemas import (
    StablishmentStageSchema,
    FPLSchema,
    ProtectionLevelSchema,
    MPAsSchema,
    MPAsTableStatsSchema,
)

In [5]:
eez_folder = Path("../data/eez_intermediate").absolute()
mpa_folder = Path("../data/mpa_intermediate").absolute()
mpaatlas_folder = Path("../data/mpaatlas_intermediate").absolute()
protectedseas_folder = Path("../data/protectedseas_intermediate").absolute()

In [6]:
location_code = pd.read_csv(eez_folder.joinpath("locations_code.csv"))
mpa_intermediate = gpd.read_file(mpa_folder.joinpath("mpa_intermediate", "mpa_intermediate.shp"))

In [7]:
protectedseas_intermediate = gpd.read_file(
    protectedseas_folder.joinpath("protectedseas_intermediate", "protectedseas_intermediate.shp")
)

In [8]:
mpaatlas_intermediate = gpd.read_file(
    mpaatlas_folder.joinpath("mpaatlas_intermediate", "mpaatlas_intermediate.shp").as_posix()
)

In [20]:
def separate_parent_iso(df: pd.DataFrame, iso_column="location_i", separator=";") -> pd.DataFrame:
    df[iso_column] = (
        df[iso_column].str.replace(" ", "").str.replace(":", separator).str.split(separator)
    )
    return df.explode(iso_column)


def calculate_area(df: pd.DataFrame, output_area_column="area_km2") -> pd.DataFrame:
    df[output_area_column] = (df.to_crs("ESRI:54009")["geometry"].area / 10**6).round(2)
    return df


def calculate_global_area(
    df: pd.DataFrame, gby_col: list, output_area_column="area_km2", iso_column="location_i"
) -> pd.DataFrame:
    global_area = (
        df.groupby([*gby_col])
        .agg({output_area_column: "sum"})
        .reset_index()
        .assign(**{iso_column: "GLOB"})
    )
    return pd.concat([global_area, df], ignore_index=True)


def add_region_iso(df: pd.DataFrame, iso_column) -> pd.DataFrame:
    regions = load_regions()

    def find_region_iso(iso: str) -> Union[str, None]:
        filtered_regions = list(filter(lambda x: iso in x["country_iso_3s"], regions.get("data")))
        return filtered_regions[0]["region_iso"] if len(filtered_regions) > 0 else None

    return df.assign(region=lambda row: row[iso_column].apply(find_region_iso))


def mpaatlas_calculation(df: pd.DataFrame, gby_col: list) -> pd.DataFrame:
    regions = (
        df.groupby([*gby_col, "region"])
        .agg({"area_km2": "sum"})
        .reset_index()
        .rename(columns={"region": "location_i"})
    )

    return pd.concat(
        [
            regions,
            df.groupby([*gby_col, "location_i"]).agg({"area_km2": "sum"}).reset_index(),
        ],
        ignore_index=True,
    )


def protectedseas_calculation(df: pd.DataFrame, gby_col: list) -> pd.DataFrame:
    regions = (
        df.groupby([*gby_col, "region"])
        .agg({"area_km2": "sum"})
        .reset_index()
        .rename(columns={"region": "iso"})
    )
    return pd.concat(
        [
            regions,
            df.groupby([*gby_col, "iso"]).agg({"area_km2": "sum"}).reset_index(),
        ],
        ignore_index=True,
    )


def batch_export(df: pd.DataFrame, batch_size: int, schema: object, folder: Path, filename: str):
    prev = 0
    for idx, size in enumerate(range(batch_size, len(df.index) + batch_size, batch_size)):
        schema(df[(df.index > prev) & (df.index < size)]).to_csv(
            folder.joinpath(f"{filename}_{idx}.csv"),
            index=True,
            encoding="utf-8",
        )
        prev = size


def fix_monaco(df: pd.DataFrame, iso_column="location_i", area_column="area_km2") -> pd.DataFrame:
    df.loc[df[iso_column] == "MCO", area_column] = 288
    return df


def set_area(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(area_km2=df[["area_km2_y", "area_km2_x"]].max(axis=1))


def output(
    df: pd.DataFrame, iso_column: str, rep_d: dict, rename: dict, drop_cols: list
) -> pd.DataFrame:
    if iso_column:
        locations_code = pd.read_csv(
            eez_folder.joinpath("locations_code.csv"), keep_default_na=False
        )
        df = df.join(locations_code.set_index("code"), on=iso_column, how="left")
    return (
        df.replace(rep_d)
        .rename(columns=rename)
        .drop(columns=drop_cols)
        .assign(
            id=df.index + 1,
        )
        .set_index("id")
    )


def filter_location(df: pd.DataFrame) -> pd.DataFrame:
    return df[~df.location.isna()]

In [10]:
mpa_atlas_table = (
    mpaatlas_intermediate.pipe(calculate_area)
)

In [12]:
test = (
    mpa_atlas_table.pipe(calculate_global_area, gby_col=["establishm"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(mpaatlas_calculation, gby_col=["establishm"])
    .pipe(fix_monaco, iso_column="location_i", area_column="area_km2")
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "establishm": {
                "actively managed": 4,
                "implemented": 6,
                "designated": 5,
                "proposed or committed": 3,
                "unknown": 1,
            }
        },
        rename={"establishm": "mpaa_establishment_stage", "area_km2": "area"},
        drop_cols=["location_i"],
    )
).assign(year=2023, protection_status=1)

StablishmentStageSchema(test[~test.location.isna()]).to_csv(
    mpaatlas_folder.joinpath("mpaatlas_stablishment.csv"), index=True
)

In [14]:
test2 = (
    mpa_atlas_table.pipe(calculate_global_area, gby_col=["protecti_1"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(mpaatlas_calculation, gby_col=["protecti_1"])
    .pipe(fix_monaco, iso_column="location_i", area_column="area_km2")
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "protecti_1": {
                "fully or highly protected": 1,
                "less protected or unknown": 2,
            }
        },
        rename={"protecti_1": "mpaa_protection_level", "area_km2": "area"},
        drop_cols=[],
    )
)
ProtectionLevelSchema(test2[~test2.location.isna()].assign(year=2023)).to_csv(
    mpaatlas_folder.joinpath("mpaatlas_protection_level.csv"), index=True
)

In [15]:
protected_seas_table = (
    protectedseas_intermediate.pipe(calculate_area)
)

In [16]:
test3 = (
    protected_seas_table.pipe(calculate_global_area, gby_col=["FPS_cat"], iso_column="iso")
    .pipe(separate_parent_iso, iso_column="iso")
    .replace(
        {
            "iso": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="iso")
    .pipe(protectedseas_calculation, gby_col=["FPS_cat"])
    .pipe(fix_monaco, iso_column="iso", area_column="area_km2")
    .pipe(
        output,
        iso_column="iso",
        rep_d={
            "FPS_cat": {
                "highly": 1,
                "moderately": 2,
                "less": 3,
            }
        },
        rename={"FPS_cat": "fishing_protection_level", "area_km2": "area"},
        drop_cols=["iso"],
    )
)

FPLSchema(test3).to_csv(protectedseas_folder.joinpath("lfp.csv"), index=True)

In [17]:
test4 = mpa_intermediate.replace(
    {
        "PARENT_ISO": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
).pipe(
    output,
    iso_column="PARENT_ISO",
    rep_d={
        "STATUS": {
            "Adopted": 4,
            "implemented": 6,
            "Established": 6,
            "Designated": 5,
            "Proposed": 3,
            "Inscribed": 3,
            "unknown": 1,
        },
        "PA_DEF": {"0": 2, "1": 1},
        "STATUS_YR": {0: pd.NA},
    },
    rename={
        "PARENT_ISO": "iso",
        "PA_DEF": "protection_status",
        "REP_M_AREA": "area",
        "STATUS_YR": "year",
        "WDPA_PID": "wdpaid",
        "NAME": "name",
        "STATUS": "mpaa_establishment_stage",
    },
    drop_cols=["geometry", "WDPAID", "iso"],
).astype({"year": "Int64"})

prev = 0
for idx, size in enumerate(range(5000, len(test4.index) + 5000, 5000)):
    MPAsSchema(test4[(test4.index > prev) & (test4.index < size)]).to_csv(
        mpa_folder.joinpath(f"mpa_{idx}.csv"),
        index=True,
        encoding="utf-8",
    )
    prev = size

In [18]:
mpa_atlas_merge = mpa_atlas_table.pipe(separate_parent_iso, iso_column="location_i").replace(
    {
        "location_i": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
)

protectedseas_merge = protected_seas_table.pipe(separate_parent_iso, iso_column="iso").replace(
    {
        "iso": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
)

In [21]:
Final = (
    test4.assign(mpa=test4.index)
    .merge(
        mpa_atlas_merge[["establishm", "wdpa_id", "protecti_1", "area_km2"]],
        left_on="wdpaid",
        right_on="wdpa_id",
        how="left",
    )
    .merge(
        protectedseas_merge[["site_id", "wdpa_id", "area_km2", "FPS_cat"]],
        left_on="wdpaid",
        right_on="wdpa_id",
        how="left",
    )
    .pipe(set_area)
    .pipe(filter_location)
    # .reset_index(drop=True)
    # .pipe(
    #     output,
    #     iso_column=None,
    #     rep_d={
    #         "protecti_1": {
    #             "fully or highly protected": 1,
    #             "less protected or unknown": 2,
    #         },
    #         "FPS_cat": {
    #             "highly": 1,
    #             "moderately": 2,
    #             "less": 3,
    #         },
    #         "establishm": {
    #             "actively managed": 4,
    #             "implemented": 6,
    #             "designated": 5,
    #             "proposed or committed": 3,
    #             "unknown": 1,
    #         },
    #     },
    #     rename={
    #         "establishm": "mpaa_establishment_stage",
    #         "protecti_1": "mpaa_protection_level",
    #         "FPS_cat": "fishing_protection_level",
    #     },
    #     drop_cols=[
    #         "wdpaid",
    #         "wdpa_id_x",
    #         "wdpa_id_y",
    #         "area_km2",
    #         "area_km2_x",
    #         "area_km2_y",
    #         "mpaa_establishment_stage",
    #         "protection_status",
    #         "name",
    #         "site_id",
    #     ],
    # )
)
# batch_export(Final, 5000, MPAsTableStatsSchema, mpa_folder, "mpa_join_mpatlas_prot")

In [22]:
protectedseas_merge.wdpa_id.unique().shape

(10786,)

In [23]:
protectedseas_merge.wdpa_id.str.replace(' ','').str.split(';').explode().unique().shape

(11984,)

In [32]:
mpa_intermediate

,WDPAID,WDPA_PID,PA_DEF,NAME,REP_M_AREA,STATUS,STATUS_YR,PARENT_ISO,geometry
0,339280,339280,1,2011181 Röskären m fl,0.00000,Designated,2012,SWE,"MULTIPOLYGON (((16.88350 58.06578, 16.88356 58..."
1,339281,339281,1,"2011182 Örskär, St. Klabben, m fl",0.00000,Designated,2012,SWE,"MULTIPOLYGON (((16.88306 58.04791, 16.88310 58..."
2,339282,339282,1,2011183 Sörö,0.00000,Designated,2012,SWE,"POLYGON ((16.77188 58.04824, 16.77193 58.04821..."
3,339283,339283,1,2011184 Tallklabben m fl,0.00000,Designated,2012,SWE,"POLYGON ((16.77969 58.03557, 16.77940 58.03529..."
4,339284,339284,1,2011185 Rotskärshällarna m fl,0.00000,Designated,2012,SWE,"POLYGON ((16.78503 58.02720, 16.78505 58.02720..."
...,...,...,...,...,...,...,...,...,...
18777,555594046,555594046,1,Mashtan Island,0.02100,Designated,2002,BHR,"POLYGON ((4759827.90237 3149355.27177, 4759827..."
18778,313506,313506,1,Arad Bay,0.44100,Designated,2003,BHR,"POLYGON ((4743197.11699 3203279.17554, 4743197..."
18779,145813,145813,1,Hawar Island and the Surrounding Terretorial Sea,51.73189,Designated,1996,BHR,"POLYGON ((4771270.71072 3129994.93264, 4771269..."
18780,145812,145812,1,Tubli Bay,15.90800,Designated,2006,BHR,"POLYGON ((4738737.37964 3195543.87956, 4738737..."


In [31]:
mpa_intermediate[mpa_intermediate.REP_M_AREA == 0]

,WDPAID,WDPA_PID,PA_DEF,NAME,REP_M_AREA,STATUS,STATUS_YR,PARENT_ISO,geometry
0,339280,339280,1,2011181 Röskären m fl,0.0,Designated,2012,SWE,"MULTIPOLYGON (((16.88350 58.06578, 16.88356 58..."
1,339281,339281,1,"2011182 Örskär, St. Klabben, m fl",0.0,Designated,2012,SWE,"MULTIPOLYGON (((16.88306 58.04791, 16.88310 58..."
2,339282,339282,1,2011183 Sörö,0.0,Designated,2012,SWE,"POLYGON ((16.77188 58.04824, 16.77193 58.04821..."
3,339283,339283,1,2011184 Tallklabben m fl,0.0,Designated,2012,SWE,"POLYGON ((16.77969 58.03557, 16.77940 58.03529..."
4,339284,339284,1,2011185 Rotskärshällarna m fl,0.0,Designated,2012,SWE,"POLYGON ((16.78503 58.02720, 16.78505 58.02720..."
...,...,...,...,...,...,...,...,...,...
18772,171913,171913,1,Osmussaare II rahnuderühm,0.0,Designated,2009,EST,"POLYGON ((1539502.98350 6807091.05073, 1539502..."
18773,145154,145154,1,Plaza Topolica,0.0,Designated,1968,MNE,"POLYGON ((1589063.89855 5022998.09694, 1589063..."
18774,145149,145149,1,Velika plaza kod Ulcinja,0.0,Designated,1968,MNE,"POLYGON ((1609288.59958 4999927.88284, 1609288..."
18775,145126,145126,1,Poluostrvo Ratac sa Zukotrljicom,0.0,Designated,1968,MNE,"POLYGON ((1586443.40372 5025234.76169, 1586443..."


In [29]:
test4[test4.area == 0]

,wdpaid,protection_status,name,area,mpaa_establishment_stage,year,location
id,,,,,,,
1,339280,1,2011181 Röskären m fl,0.0,5,2012,144.0
2,339281,1,"2011182 Örskär, St. Klabben, m fl",0.0,5,2012,144.0
3,339282,1,2011183 Sörö,0.0,5,2012,144.0
4,339283,1,2011184 Tallklabben m fl,0.0,5,2012,144.0
5,339284,1,2011185 Rotskärshällarna m fl,0.0,5,2012,144.0
...,...,...,...,...,...,...,...
18773,171913,1,Osmussaare II rahnuderühm,0.0,5,2009,53.0
18774,145154,1,Plaza Topolica,0.0,5,1968,107.0
18775,145149,1,Velika plaza kod Ulcinja,0.0,5,1968,107.0


In [27]:
Final[Final.area == 0]

,wdpaid,protection_status,name,area,mpaa_establishment_stage,year,location,mpa,establishm,wdpa_id_x,protecti_1,area_km2_x,site_id,wdpa_id_y,area_km2_y,FPS_cat,area_km2
0,339280,1,2011181 Röskären m fl,0.0,5,2012,144.0,1,NaN,NaN,NaN,NaN,AISWE696,339280,0.67,less,0.67
1,339281,1,"2011182 Örskär, St. Klabben, m fl",0.0,5,2012,144.0,2,NaN,NaN,NaN,NaN,AISWE691,339281,0.49,less,0.49
2,339282,1,2011183 Sörö,0.0,5,2012,144.0,3,NaN,NaN,NaN,NaN,AISWE710,339282,0.32,less,0.32
3,339283,1,2011184 Tallklabben m fl,0.0,5,2012,144.0,4,NaN,NaN,NaN,NaN,AISWE721,339283,0.20,less,0.20
4,339284,1,2011185 Rotskärshällarna m fl,0.0,5,2012,144.0,5,NaN,NaN,NaN,NaN,AISWE698,339284,0.20,less,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18918,171913,1,Osmussaare II rahnuderühm,0.0,5,2009,53.0,18773,NaN,NaN,NaN,NaN,AIEST278,171913,0.00,less,0.00
18919,145154,1,Plaza Topolica,0.0,5,1968,107.0,18774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18920,145149,1,Velika plaza kod Ulcinja,0.0,5,1968,107.0,18775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18921,145126,1,Poluostrvo Ratac sa Zukotrljicom,0.0,5,1968,107.0,18776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
test_Final = Final[~Final.area_km2.isna()][
    ["wdpaid", "name", "area_km2_x", "area", "area_km2_y", "FPS_cat", "area_km2", "protecti_1"]
]
test_Final["area_km2_1"] = test_Final.area_km2_y - test_Final.area_km2_x
test_Final[(test_Final.area_km2_1 != 0) & (test_Final.area_km2_1.notna())]

,wdpaid,name,area_km2_x,area,area_km2_y,FPS_cat,area_km2,protecti_1,area_km2_1
184,345888,Terres Australes Françaises,545381.47,0.000000,1646295.08,moderately,1646295.08,less protected or unknown,1100913.61
185,345888,Terres Australes Françaises,989240.07,0.000000,1646295.08,moderately,1646295.08,less protected or unknown,657055.01
186,345888,Terres Australes Françaises,120379.58,0.000000,1646295.08,moderately,1646295.08,fully or highly protected,1525915.50
398,354086_B,Macquarie Island,104492.63,104758.767956,104492.64,moderately,104492.64,less protected or unknown,0.01
1416,400012,Rose Atoll Marine National Monument,158.62,0.000000,35003.99,highly,35003.99,fully or highly protected,34845.37
...,...,...,...,...,...,...,...,...,...
15490,555636626,Refúgio De Vida Silvestre Do Arquipélago De Al...,669.41,674.724049,677.70,highly,677.70,fully or highly protected,8.29
15491,555636626,Refúgio De Vida Silvestre Do Arquipélago De Al...,6.31,674.724049,677.70,highly,677.70,fully or highly protected,671.39
16157,555655556,Big Bend Seagrasses Aquatic Preserve,2764.39,0.000000,3997.65,less,3997.65,less protected or unknown,1233.26
16160,555655559,Biscayne Bay Aquatic Preserve,279.83,0.000000,262.57,less,279.83,less protected or unknown,-17.26
